## 트리의 앙상블
+ 앙상블 학습은 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘

## 랜덤 포레스트
+ 랜덤 포레스트는 앙상블 학습의 대표 주자중 하나로 안정적인 성능 덕분에 널리 사용
+ 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만들고, 각 결정 트리의 예측을 사용해 최종 예측을 만든다

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [15]:
wine = pd.read_csv("https://bit.ly/wine_csv_data")

In [16]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [17]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [18]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [19]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [20]:
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)

In [22]:
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [24]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


### OOB(out of bag) 샘플
+ 랜덤 포레스트는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들고 결정 트리를 훈련시킴
+ 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플을 OOB샘플이라 부른다
+ 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다 -> 검증 세트의 역할

In [27]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리
+ 기본적으로 랜덤 포레스트와 매우 비슷하게 동작
+ 랜덤 포레스트와의 차이는 부트스트랩 샘플을 사용하지 않음 -> 전체 샘플 사용
+ 대신 도르를 분할할 때 가장 좋은 분할을 찾는 것이 아니라, 무작위로 분할

In [28]:
from sklearn.ensemble import ExtraTreesClassifier

In [29]:
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [30]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그래디언트 부스팅
+ 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
+ 경사하강법을 사용하여 트리를 앙상블에 추가

In [31]:
from sklearn.ensemble import GradientBoostingClassifier

In [32]:
gb =GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [33]:
gb =GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [34]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15882696 0.6799705  0.16120254]


## 히스토그램 기반 그래디언트 부스팅
+ 히스토그램 기반 그래디언트 부스팅은 입력 특성을 256개의 구간으로 나눈다
+ 256개의 구간중 하나를 떼어 놓고 누락된 값을 위해 사용하기 때문에, 입력에 누락된 특성이 있더라도 이를 따로 전처리 할 필요가 없다

In [35]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [36]:
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [37]:
from sklearn.inspection import permutation_importance # 특성을 하나씩 랜덤하게 섞어 모델의 성능이 변화하는지 관찰 -> 중요한 특성 계산

In [38]:
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [39]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [40]:
hgb.score(test_input, test_target)

0.8723076923076923

In [ ]:
from xgboost import XGBClassifier # 다양한 부스팅 알고리즘을 지원하는 라이브러리

In [42]:
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)

In [43]:

scores = cross_validate(xgb, train_input, train_target, return_train_score = True)

In [44]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [45]:
from lightgbm import LGBMClassifier

In [46]:
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)

In [47]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
